In [1]:
import glob
import os
import tempfile
import warnings

import geopandas as gpd
import pandas as pd
from gplately.tools import plate_isotherm_depth

from lib.assign_regions import assign_regions
from lib.calculate_convergence import run_calculate_convergence
from lib.check_files import (
    check_source_data,
    check_plate_model,
)
from lib.combine_point_data import combine_point_data
from lib.coregister_combined_point_data import run_coregister_combined_point_data
from lib.coregister_crustal_thickness import run_coregister_crustal_thickness
from lib.coregister_magnetic import coregister_magnetic
from lib.coregister_ocean_rasters import run_coregister_ocean_rasters
from lib.create_study_area_polygons import run_create_study_area_polygons
from lib.erodep import calculate_erodep
from lib.generate_unlabelled_points import generate_unlabelled_points
from lib.misc import (
    calculate_slab_flux,
    calculate_water_thickness,
)
from lib.slab_dip import calculate_slab_dip

# Suppress occasional joblib warnings
%env PYTHONWARNINGS=ignore::UserWarning

warnings.simplefilter("ignore", UserWarning)


env: PYTHONWARNINGS=ignore::UserWarning


In [2]:
random_seed = 1234

n_jobs = int(os.environ.get("N_JOBS", 8))
times = range(171)
verbose = False

num_unlabelled = 200  # per timestep


### Input and output files

In [3]:
model_dir = "plate_model"
check_plate_model(model_dir, verbose=verbose)

data_dir = "source_data"
check_source_data(data_dir, verbose=verbose)
deposits_filename = os.path.join(data_dir, "deposit_data_global.csv")
regions_filename = os.path.join(data_dir, "regions.shp")
agegrid_dir = os.path.join(data_dir, "AgeGrids")
sedthick_dir = os.path.join(data_dir, "SedimentThickness")
carbonate_dir = os.path.join(data_dir, "CarbonateThickness")
co2_dir = os.path.join(data_dir, "CrustalCO2")
crustal_thickness_dir = os.path.join(data_dir, "CrustalThickness")
mag_anomaly_filename = os.path.join(
    data_dir,
    "MagneticAnomaly",
    "emag2_upcont_interpolated.nc",
)
subducted_quantities_dir = os.path.join(
    data_dir,
    "SubductedQuantities",
)
erodep_dir = os.path.join(data_dir, "ErosionDeposition")

output_dir = "extracted_data"
os.makedirs(output_dir, exist_ok=True)
subduction_data_filename = os.path.join(output_dir, "subducting_plate_data.csv")
study_area_dir = os.path.join(output_dir, "study_area_polygons")
output_filename = os.path.join(output_dir, "training_data_global.csv")

feature_filenames = glob.glob(
    os.path.join(
        model_dir,
        "*.gpml",
    )
)
rotation_filenames = glob.glob(
    os.path.join(
        model_dir,
        "*.rot",
    )
)
static_polygons_filename = os.path.join(
    model_dir,
    "StaticGeometries",
    "StaticPolygons",
    "Clennett_2020_StaticPolygons.gpml",
)


### Subducting plate data

In [4]:
if subduction_data_filename is not None and os.path.isfile(subduction_data_filename):
    subduction_data = pd.read_csv(subduction_data_filename)
else:
    with tempfile.TemporaryDirectory() as subduction_kinematics_dir:
        run_calculate_convergence(
            nprocs=n_jobs,
            min_time=min(times),
            max_time=max(times),
            topology_filenames=feature_filenames,
            rotation_filenames=rotation_filenames,
            output_dir=subduction_kinematics_dir,
            verbose=verbose,
        )

        subduction_data = run_coregister_ocean_rasters(
            nprocs=n_jobs,
            times=times,
            input_data=subduction_kinematics_dir,
            agegrid_dir=agegrid_dir,
            topology_features=feature_filenames,
            rotation_model=rotation_filenames,
            sedthick_dir=sedthick_dir,
            carbonate_dir=carbonate_dir,
            co2_dir=co2_dir,
            subducted_thickness_dir=os.path.join(
                subducted_quantities_dir,
                "plate_thickness",
            ),
            subducted_sediments_dir=os.path.join(
                subducted_quantities_dir,
                "sediment_thickness",
            ),
            subducted_carbonates_dir=os.path.join(
                subducted_quantities_dir,
                "carbonate_thickness",
            ),
            subducted_water_dir=os.path.join(
                subducted_quantities_dir,
                "water_thickness",
            ),
            verbose=verbose,
        )
    subduction_data["plate_thickness (m)"] = plate_isotherm_depth(
        subduction_data["seafloor_age (Ma)"],
        maxiter=100,
    )
    subduction_data = calculate_water_thickness(subduction_data)
    subduction_data = calculate_slab_flux(subduction_data)
    subduction_data = calculate_slab_dip(subduction_data)
    if subduction_data_filename is not None:
        subduction_data.to_csv(subduction_data_filename, index=False)


### Create study area polygons along subduction zones

In [5]:
if not os.path.isdir(study_area_dir):
    run_create_study_area_polygons(
        nprocs=n_jobs,
        times=times,
        topological_features=feature_filenames,
        rotation_model=rotation_filenames,
        output_dir=study_area_dir,
        verbose=verbose,
        return_output=False,
    )


### Generate random unlabelled data points

In [6]:
unlabelled = generate_unlabelled_points(
    times=times,
    input_dir=study_area_dir,
    num=num_unlabelled,
    threads=n_jobs,
    seed=random_seed,
    topological_features=feature_filenames,
    rotation_model=rotation_filenames,
    verbose=verbose,
)


### Combine labelled deposit/non-deposit data with random unlabelled data

In [7]:
combined_points = combine_point_data(
    deposit_data=deposits_filename,
    unlabelled_data=unlabelled,
    static_polygons=static_polygons_filename,
    topological_features=feature_filenames,
    rotation_model=rotation_filenames,
    study_area_dir=study_area_dir,
    min_time=min(times),
    max_time=max(times),
    n_jobs=n_jobs,
    verbose=verbose,
)
del unlabelled


### Assign subduction data to point deposit/non-deposit/unlabelled data

In [8]:
coregistered_data = run_coregister_combined_point_data(
    point_data=combined_points,
    subduction_data=subduction_data,
    n_jobs=n_jobs,
    verbose=verbose,
)
del combined_points, subduction_data


### Assign magnetic anomaly and crustal thickness data to point data

In [9]:
coregistered_data = run_coregister_crustal_thickness(
    point_data=coregistered_data,
    input_dir=crustal_thickness_dir,
    n_jobs=n_jobs,
    verbose=verbose,
)
coregistered_data = coregister_magnetic(
    data=coregistered_data,
    filename=mag_anomaly_filename,
    n_jobs=n_jobs,
)


### Calculate cumulative erosion

In [10]:
coregistered_data = calculate_erodep(
    coregistered_data,
    input_dir=erodep_dir,
    n_jobs=n_jobs,
    column_name="erosion (m)",
    verbose=verbose,
)


banana
(38001, 57)
(38001,)
0.0 (389, 57)
1.0 (226, 57)
extract erodep TIME: 1.0
source_data/ErosionDeposition
['source_data/ErosionDeposition/erodep20Ma.nc', 'source_data/ErosionDeposition/erodep155Ma.nc', 'source_data/ErosionDeposition/erodep85Ma.nc', 'source_data/ErosionDeposition/erodep120Ma.nc', 'source_data/ErosionDeposition/erodep110Ma.nc', 'source_data/ErosionDeposition/erodep130Ma.nc', 'source_data/ErosionDeposition/erodep65Ma.nc', 'source_data/ErosionDeposition/erodep15Ma.nc', 'source_data/ErosionDeposition/erodep60Ma.nc', 'source_data/ErosionDeposition/erodep135Ma.nc', 'source_data/ErosionDeposition/erodep90Ma.nc', 'source_data/ErosionDeposition/erodep50Ma.nc', 'source_data/ErosionDeposition/erodep25Ma.nc', 'source_data/ErosionDeposition/erodep170Ma.nc', 'source_data/ErosionDeposition/erodep100Ma.nc', 'source_data/ErosionDeposition/erodep10Ma.nc', 'source_data/ErosionDeposition/erodep40Ma.nc', 'source_data/ErosionDeposition/erodep45Ma.nc', 'source_data/ErosionDeposition/erod

POINT:ERODEP [-203.1027985  -203.1027985  -203.73902193 -203.73902193 -213.45491342
 -213.45491342 -218.7051428  -218.7051428  -160.02574664 -160.02574664
 -196.94014501 -196.94014501 -172.4747566  -172.4747566  -162.33825859
 -162.33825859 -156.22559735 -156.22559735 -159.52727697 -159.52727697
 -156.71857537 -156.71857537 -156.90413799 -156.90413799 -164.29053541
 -272.39046266 -272.39046266 -233.7617885  -233.7617885  -171.57209732
 -171.57209732 -170.17725948 -170.17725948 -170.43644607 -170.43644607
 -169.19255104 -169.19255104 -169.78215231 -169.78215231 -169.84075126
 -169.84075126 -168.95366311 -168.95366311 -169.43706978 -169.43706978
 -169.03443723 -169.03443723 -172.62893641 -172.62893641 -172.76589396
 -172.76589396 -148.29423744 -286.62418239 -139.65512334 -144.3793937
 -144.3793937  -148.55598575 -148.55598575 -175.28383604 -175.28383604
 -137.54529606 -141.28794775 -127.71829855 -139.95943097 -139.16508639
 -139.16765244 -231.41084397 -135.43416362 -183.50143498 -135.283

POINT:ERODEP [ -326.17155157  -346.73648173  -345.7335939   -404.61716439
  -188.51752998  -188.51752998  -193.34700404  -193.34700404
  -187.4024654   -187.4024654   -151.38115692  -151.38115692
    49.86398585    49.86398585  -171.76501343  -171.76501343
  -146.30511432  -146.30511432  -145.27559068  -145.27559068
  -138.44244944  -138.44244944  -110.96738464  -110.96738464
   -68.50116633   -68.50116633  -565.44892677  -175.38211728
  -175.38211728  -112.5149769   -220.15642834  -220.15642834
  -321.16372093  -226.39927065  -705.71920026  -230.38124793
   -31.6753031  -1066.1291129   -864.75714952  -354.28152257
  -132.54329499  -103.09348637  -126.57223292  -126.57223292
  -244.51578074  -244.51578074   -83.12427867  -271.06988409
  -271.06988409  -218.51761521            nan  -289.99173177
   -33.03617345   -54.69320445   -99.61456016  -137.21985021
  -178.70448406  -178.70448406   -30.33961694  -411.42322148
  -411.42322148   -25.07963442   -50.82671845  -236.92007955
  -255.1630

[[          nan           nan           nan ...           nan
            nan           nan]
 [-161.75743554 -243.30065261 -279.92747578 ...  -37.30088264
   -37.47333524 -161.75743554]
 [ -55.09193047  -85.25669375 -102.63244769 ...  -37.47836707
   -37.23025552  -55.09193047]
 ...
 [          nan           nan           nan ...           nan
            nan           nan]
 [          nan           nan           nan ...           nan
            nan           nan]
 [          nan           nan           nan ...           nan
            nan           nan]]
(1801,) (3601,)
POINT:ERODEP [-113.21516365]
3.7 (1, 57)
extract erodep TIME: 4.0
extract erodep timestep_total: [[         nan          nan          nan ...          nan          nan
           nan]
 [-40.43935889 -60.82516315 -69.98186895 ...  -9.32522066  -9.36833381
  -40.43935889]
 [-13.77298262 -21.31417344 -25.65811192 ...  -9.36959177  -9.30756388
  -13.77298262]
 ...
 [         nan          nan          nan ...          nan

extract erodep timestep_total: [[         nan          nan          nan ...          nan          nan
           nan]
 [-40.43935889 -60.82516315 -69.98186895 ...  -9.32522066  -9.36833381
  -40.43935889]
 [-13.77298262 -21.31417344 -25.65811192 ...  -9.36959177  -9.30756388
  -13.77298262]
 ...
 [         nan          nan          nan ...          nan          nan
           nan]
 [         nan          nan          nan ...          nan          nan
           nan]
 [         nan          nan          nan ...          nan          nan
           nan]]
extract erodep TIME: 4.0
extract erodep timestep_total: [[         nan          nan          nan ...          nan          nan
           nan]
 [-40.43935889 -60.82516315 -69.98186895 ...  -9.32522066  -9.36833381
  -40.43935889]
 [-13.77298262 -21.31417344 -25.65811192 ...  -9.36959177  -9.30756388
  -13.77298262]
 ...
 [         nan          nan          nan ...          nan          nan
           nan]
 [         nan          nan     

...

POINT:ERODEP [ -5283.36685332             nan  -8460.51138362  -2120.91563766
   -486.14110698  -8497.31943952  -4201.31155373  -3525.07898348
  -7157.72939121  -2005.6843671   -3033.61326168  -4163.22574527
   -448.78174363  -9954.64568628  -8492.55164167             nan
  -1298.89192247  -8563.79563049  -4999.23110748             nan
             nan  -3565.67606396  -2060.33133812  -8568.66522197
  -1107.43584004  -5611.86886357  -2079.90078025  -4963.07427506
  -3133.10899588             nan  -6649.03478895  -3865.82460894
             nan  -5558.1516063              nan  -6011.77687539
  -2309.97321211   -938.86869965  -3557.22599943  -4462.79829933
  -7599.22250835   -480.75936103  -8140.90376669             nan
  -6046.12566494 -11761.88050959  -1906.26477693  -5816.22114959
  -3222.38784014  -9776.69697073             nan  -6986.91866123
  -2010.40210828  -2496.65277811  -5117.97148648  -5538.20892559
  -4138.94517665  -1796.76812236             nan  -5665.60736906
  -5272.4796

POINT:ERODEP [ -1577.88185606             nan  -6424.17575505  -2542.92404324
  -8613.66912338  -1577.92575977 -13690.90798647  -5429.7951893
  -2668.2171765   -2291.99566218  -5211.62495238   -739.19914391
  -1267.70785553   -934.75372474  -4170.32929891  -2016.91908254
  -2971.14422706  -2626.68966601  -1928.44311434  -6991.3236588
  -1962.9222533   -2507.20566645  -9268.10263583  -4790.70148306
  -1936.43974423  -4884.8449946    -179.38363629   -179.38363629
  -6812.79272664  -2724.36338163  -5236.93740985  -1615.75751233
  -2673.78530951   -435.1322215   -4064.40700729  -2714.47322503
   -458.1442256   -3396.40106143  -2051.22957865  -6409.93169219
  -7460.36317998  -2593.00894694  -8617.86242111  -1096.48968951
             nan  -4030.27798732  -7483.03009815  -1548.51367935
             nan  -2609.26140317  -2469.99486361  -4087.61888185
  -8519.3386504    -482.9286211   -5034.17191029  -6103.23345974
  -1965.71370098  -4402.49109895             nan  -2451.48835003
  -3594.342856

POINT:ERODEP [ -6667.63418451  -6592.09235042  -2071.87003232  -3500.28950681
   -745.03240036             nan  -2029.62942588  -6997.33059337
  -7781.7511628   -3397.78393828  -1498.0464733              nan
  -1974.73045682 -10291.24391723  -2258.8183742   -1945.02362236
  -3766.55840096  -3314.62006469  -3081.65877888   -394.0647824
  -7782.16962031  -2299.36702402  -1480.76484968  -6638.63762151
  -3631.96405029  -1783.20720253  -1862.87871292  -2184.44693175
  -5805.86592298  -4959.23029646  -3531.88435819   -377.43917003
  -6066.46923122  -7904.6164044   -3941.64159321   -526.21639012
  -6707.32075919  -5339.34888751  -4678.95034654   -796.0407716
  -6349.04589119  -1654.07325187   -450.88144434  -1905.54788156
  -3018.24539361   -555.93213677  -1442.56731785  -2528.62839037
 -11812.2702057    -453.74349263  -2325.65075152  -1088.63170624
             nan  -1933.89623373   -534.67697744  -7333.06201857
  -7333.06201857  -2263.65109057  -2266.49133109             nan
  -2010.809039

POINT:ERODEP [ -4833.22723391  -5009.06874611  -5059.97536667  -4941.25968747
  -4999.84993358  -9237.45301911  -1979.98188879  -4845.56570476
   -476.36713944  -7827.59035443  -3364.22353744  -2098.83751044
   -589.44331135  -1452.13610066  -1374.33701374  -4897.0901152
             nan   -551.73889868             nan  -3173.91007463
  -4062.84356077  -1021.11543263  -3515.13406979  -4745.55253112
  -7037.09845307   -921.57743343    -34.57267939  -3532.58165272
   -727.75110338  -4377.82700429  -3286.39339276   -591.89434923
  -1465.95652683  -1965.84217473  -5219.53264821  -6975.24001611
  -6941.13396322  -1208.39781348   -330.76159442  -6409.74346464
  -5114.95879099  -3783.80336356  -1116.97895227  -2914.55845431
  -1065.68322373 -11544.34689285  -2029.11693852  -4167.91565448
  -2992.39940415  -2361.77800745  -8301.74286347  -5637.17307023
  -5989.36003442  -5975.59363656  -4993.31776772 -11726.32903288
  -2034.01298431  -2931.10687235  -1622.63968712  -2031.35577615
  -7420.24991

POINT:ERODEP [ -1210.99157335  -5187.48452582  -3479.70235989  -6742.37027375
  -8179.74523698  -6581.76506192  -5611.47334529  -2155.96393399
  -2042.83512272  -3291.3910594   -1944.13483678  -4834.99977901
   -469.52699036  -6002.56924565  -5174.32685241  -7481.51389247
  -1499.11435017  -7101.20889236  -5340.36779815  -6209.89782673
  -7585.66236247             nan   -114.11583918  -2075.61566217
  -3262.80173468   -485.29118675  -1456.50158894  -3684.36237815
  -2164.44838765  -5694.99662802  -1523.29276377  -7278.32748998
 -10089.63125403  -3202.04817419  -8305.0336788  -17896.58517499
  -5760.58262536  -8130.76401472  -2299.5338912   -2534.09513792
             nan  -4317.48257119   -603.36960027  -3703.16002653
             nan  -6655.38889533  -1394.42855419  -2080.78639524
  -4271.63975255  -2621.73001641   -728.21559376   -584.72867656
   -568.95647704   -914.35511549  -1761.48474657   -469.97344973
  -2488.73804137  -2597.30787395  -4607.70485631  -6302.744193
  -8581.987854

### Assign data to regions

In [11]:
points = gpd.GeoSeries.from_xy(
    coregistered_data["present_lon"],
    coregistered_data["present_lat"],
    index=coregistered_data.index,
)
coregistered_data["region"] = assign_regions(
    points,
    regions=regions_filename,
)
del points


In [12]:
coregistered_data.to_csv(output_filename, index=False)

coregistered_data.groupby(["source", "region", "label"]).size()


source                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   region          label     
COULTHARD D. A. JR., REAGAN M. K., GIORDANO D., BINDEMAN I. N., BROUNCE M. N., ALMEEV R. R., RYAN J. G., CHAPMAN T.,

In [13]:
coregistered_data

,present_lon,present_lat,age (Ma),label,source,plate_id,lon,lat,overriding_plate_id,region,...,crustal_thickness_n,crustal_thickness_range (m),magnetic_anomaly_mean (nT),magnetic_anomaly_min (nT),magnetic_anomaly_max (nT),magnetic_anomaly_median (nT),magnetic_anomaly_std (nT),magnetic_anomaly_n,magnetic_anomaly_range (nT),erosion (m)
0,-153.091800,60.031900,0.0,negative,Diaz-Rodriguez et al. 2021,1073.0,-153.091800,60.031900,101,North America,...,224,3295.085938,154.652344,35.811516,282.693878,149.069107,62.332241,57,246.882362,-0.000000
1,-153.435000,59.362600,0.0,negative,Diaz-Rodriguez et al. 2021,1073.0,-153.435000,59.362600,101,North America,...,222,3179.992188,73.226997,67.446449,90.968697,71.927338,5.519875,58,23.522247,-0.000000
2,-153.535100,58.859600,0.0,negative,Diaz-Rodriguez et al. 2021,1073.0,-153.535100,58.859600,101,North America,...,219,2803.796875,137.284744,122.035904,156.593994,135.497070,9.474295,57,34.558090,-0.000000
3,-153.673800,58.770300,0.0,negative,Diaz-Rodriguez et al. 2021,1073.0,-153.673800,58.770300,101,North America,...,219,2763.070312,134.857651,88.042747,158.585251,144.321899,20.285322,56,70.542503,-0.000000
4,-154.357300,58.452800,0.0,negative,Diaz-Rodriguez et al. 2021,1073.0,-154.357300,58.452800,101,North America,...,216,2659.718750,34.768494,23.034136,52.113960,34.099560,7.306161,53,29.079824,-0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37996,-179.307731,-46.634435,170.0,unlabelled,random,NaN,-111.255256,-81.730617,801,Other,...,640,2281.203125,-47.745644,-55.562462,-26.274920,-50.548042,7.527466,44,29.287542,NaN
37997,37.250119,36.488104,170.0,unlabelled,random,NaN,43.550673,27.166732,301,Tethys,...,76,2200.000000,-89.174515,-128.204422,-65.533401,-85.494110,15.364982,34,62.671021,575.104159
37998,139.559378,69.106330,170.0,unlabelled,random,NaN,-142.628548,75.011185,101,Other,...,0,NaN,75.784271,9.306231,106.699234,80.302704,22.314474,80,97.393003,2610.979475
37999,142.512800,53.847479,170.0,unlabelled,random,NaN,-162.534245,69.316840,926,East Asia,...,0,NaN,72.711830,39.225700,106.439354,72.116463,20.911398,46,67.213654,1955.075798
